In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
from shutil import copy  
np.random.seed(12345)    

In [2]:
df = pd.read_csv('both_added_2_18_2022.csv')
df

,Unnamed: 0,ID,Dataset,Histology,Data Availability,Cohort,Age,TobaccoUse,Gender,Unnamed: 9
0,0,Hugo_LUNG1-001,NSCLC Radiomics,large cell,Yes,NaN,78.751500,NaN,Male,NaN
1,1,Hugo_LUNG1-002,NSCLC Radiomics,squamous cell carcinoma,Yes,NaN,83.800100,NaN,Male,NaN
2,2,Hugo_LUNG1-003,NSCLC Radiomics,large cell,Yes,NaN,68.180700,NaN,Male,NaN
3,3,Hugo_LUNG1-004,NSCLC Radiomics,squamous cell carcinoma,Yes,NaN,70.880200,NaN,Male,NaN
4,4,Hugo_LUNG1-005,NSCLC Radiomics,squamous cell carcinoma,Yes,NaN,80.481900,NaN,Male,NaN
...,...,...,...,...,...,...,...,...,...,...
1912,1956,Patient_1294,Driver-negative,Adenocarcinoma,Yes,test,52.043836,Former,Female,NaN
1913,1957,Patient_1295,Driver-positive,Squamous cell carcinoma,Yes,independent,68.123288,Former,Male,NaN
1914,1958,Patient_1296,Driver-negative,Adenocarcinoma,Yes,test,72.583562,Current,Female,NaN
1915,1959,Patient_1297,Driver-negative,Adenocarcinoma,Yes,test,64.602740,Never (<100 lifetime cigarettes),Female,NaN


In [3]:
set(df["TobaccoUse"].tolist())

{'Current', 'Former', 'Never (<100 lifetime cigarettes)', 'Nonsmoker', nan}

In [4]:
set(df["Gender"].tolist())

{'Female', 'Male', nan}

In [5]:
df["ID"].tolist()
count = 0
IDs = []
for i in df["ID"].tolist():
    Age = df[df["ID"]==i]['Age']   #TobaccoUse    Gender
    TobaccoUse = df[df["ID"]==i]['TobaccoUse']
    Gender = df[df["ID"]==i]['Gender']
    #print(Age )
    #print(TobaccoUse)
    #print(Gender)
    if not pd.isnull(TobaccoUse.item()) and not pd.isnull(Gender.item()) and not pd.isnull(Age.item()): #pd.isnull  TobaccoUse.item()=='NaN' or
        #print(count)
        IDs.append(i)
        count += 1
        
print(len(IDs))

1284


In [21]:
# shuffling
IDs = np.array(IDs)
indices_A = np.arange(len(IDs))
np.random.shuffle(indices_A)
IDs = IDs[indices_A]

# train, val
n_train = 1000
n_val = 284

IDs_train = IDs[:n_train]
IDs_val = IDs[n_train:]

print(len(IDs_train), len(IDs_val))

1000 284


In [22]:
age_array = []
for i, id in enumerate(IDs_train):
    Age = df[df["ID"]==id]['Age'].item()  #TobaccoUse    Gender
    age_array.append(Age)

age_array = np.array(age_array)

age_mean = np.mean(age_array)
age_std = np.std(age_array)
print(age_mean, age_std)
age_array = (age_array -age_mean)/age_std
print(age_array.mean(), age_array.std())

65.408339726 10.294647007845224
5.719869022868807e-16 1.0


In [33]:
age_array = []
for i, id in enumerate(IDs_val):
    Age = df[df["ID"]==id]['Age']   #TobaccoUse    Gender
    Age = (Age.item()-age_mean)/age_std
    age_array.append(Age)

age_array = np.array(age_array)

np.mean(age_array)
np.std(age_array)

print(np.mean(age_array), np.std(age_array))
print(age_array.min(), age_array.max())

0.043748119282224794 0.9522283462009385
-3.3609817334807444 3.2316185857220003


In [25]:

train_folder = 'E:/data/Histology/Nature_Gemini_4d_Feb2022/TrainValTest/NatureGeminiAll_ageN'
val_folder =   'E:/data/Histology/Nature_Gemini_4d_Feb2022/TrainValTest/NatureGeminiAll_ageN'
test_folder =  'E:/data/Histology/Nature_Gemini_4d_Feb2022/TrainValTest/NatureGeminiAll_ageN'

'''
train_folder = 'C:/Users/MSalehjahromi/Desktop/NatureGeminiAll'
val_folder =   'C:/Users/MSalehjahromi/Desktop/NatureGeminiAll'
test_folder =  'C:/Users/MSalehjahromi/Desktop/NatureGeminiAll'
'''

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)


nature_train = ['TCGA LUAD/LUSC','NSCLC Radiomics','NSCLC Radiomics-Genomics']
nature_val = ['Stanford','NSCLC Radiogenomics']
nature_test = [] #['NSCLC Radiogenomics'] #Sandys

df = pd.read_csv('both_added_2_18_2022.csv')
df_train = pd.DataFrame(columns=['ID', 'Histology','Histology012', 'Age', 'gender_cat', 'gender', 'TobaccoUse', 'smoking' ])
df_val   = pd.DataFrame(columns=['ID', 'Histology','Histology012', 'Age', 'gender_cat', 'gender', 'TobaccoUse', 'smoking' ])
df_test  = pd.DataFrame(columns=['ID', 'Histology','Histology012', 'Age', 'gender_cat', 'gender', 'TobaccoUse', 'smoking' ])

D_hist = {   'adenocarcinoma'                   : 1 ,
             'Adenocarcinoma'                   : 1 ,
             'Squamous cell carcinoma'          : 0 ,
             'squamous cell carcinoma'          : 0 ,
             'Sarcomatoid carcinoma'            : 1 ,
             'Adenosquamous carcinoma'          : 1 ,
             'Large cell carcinoma'             : 1 ,
             'Non-small cell carcinoma NOS'     : 1 ,
             'Sarcomatoid carcinoma'            : 1 ,
             'large cell'                       : 1 ,
             'nos'                              : 1 }

D_gender = {   'Female'                         : 0 ,
               'Male'                           : 1 }

D_smoking = {'Current'                          : 0 ,
             'Former'                           : 0 ,
             'Never (<100 lifetime cigarettes)' : 1 , 
             'Nonsmoker'                        : 1 }


for i, id in enumerate(IDs_train):
    Hist = df[df["ID"]==id]['Histology']
    Age = df[df["ID"]==id]['Age']   #TobaccoUse    Gender
    Age = (Age.item()-age_mean)/age_std
    TobaccoUse = df[df["ID"]==id]['TobaccoUse']
    Gender = df[df["ID"]==id]['Gender']
    if id.startswith('Patient'):
        idd = 'CT_'+id[8:]
    else:
        idd = id
    #'ID', 'Histology','Histology012', 'Age', 'gender_cat', 'gender', 'TobaccoUse', 'smoking' 
    series_id = [pd.Series([idd, Hist.item(), D_hist[Hist.item()], Age, Gender.item(), D_gender[Gender.item()], TobaccoUse.item(), D_smoking[TobaccoUse.item()]], index = df_train.columns[0:8])]
    df_train = df_train.append(series_id, ignore_index=True)

    
for i, id in enumerate(IDs_val):
    Hist = df[df["ID"]==id]['Histology']
    Age = df[df["ID"]==id]['Age']   #TobaccoUse    Gender
    Age = (Age.item()-age_mean)/age_std
    TobaccoUse = df[df["ID"]==id]['TobaccoUse']
    Gender = df[df["ID"]==id]['Gender']
    if id.startswith('Patient'):
        idd = 'CT_'+id[8:]
    else:
        idd = id
    #'ID', 'Histology','Histology012', 'Age', 'gender_cat', 'gender', 'TobaccoUse', 'smoking' 
    series_id = [pd.Series([idd, Hist.item(), D_hist[Hist.item()], Age, Gender.item(), D_gender[Gender.item()], TobaccoUse.item(), D_smoking[TobaccoUse.item()]], index = df_val.columns[0:8])]
    df_val = df_val.append(series_id, ignore_index=True)

df_train.to_csv(os.path.join(train_folder, 'train.csv'))
df_val.to_csv(os.path.join(val_folder, 'val.csv'))
#df_test.to_csv(os.path.join(test_folder, 'test.csv'))

In [31]:
Gemini_l = 0
Nature_l = 0
for i, id in enumerate(IDs):
    if id.startswith("Patient"):
        Gemini_l += 1
    elif id.startswith("Sand"):
        Nature_l += 1

print(Gemini_l,Nature_l, Gemini_l+Nature_l)

Gemini_l = 0
Nature_l = 0
for i, id in enumerate(IDs_train):
    if id.startswith("Patient"):
        Gemini_l += 1
    elif id.startswith("Sand"):
        Nature_l += 1

print(Gemini_l,Nature_l, Gemini_l+Nature_l)

Gemini_l = 0
Nature_l = 0
for i, id in enumerate(IDs_val):
    if id.startswith("Patient"):
        Gemini_l += 1
    elif id.startswith("Sand"):
        Nature_l += 1

print(Gemini_l,Nature_l, Gemini_l+Nature_l)

1086 198 1284
838 162 1000
248 36 284


In [26]:
df_train

,ID,Histology,Histology012,Age,gender_cat,gender,TobaccoUse,smoking
0,Sandy1_R01-030,adenocarcinoma,1,2.000230,Male,1,Former,0
1,Sandy1_R01-108,squamous cell carcinoma,0,1.125989,Male,1,Former,0
2,CT_0750,Adenocarcinoma,1,1.383338,Male,1,Former,0
3,CT_0202,Squamous cell carcinoma,0,0.093666,Male,1,Former,0
4,CT_1232,Non-small cell carcinoma NOS,1,0.899245,Male,1,Never (<100 lifetime cigarettes),1
...,...,...,...,...,...,...,...,...
995,CT_0616,Adenocarcinoma,1,1.503895,Male,1,Former,0
996,CT_1076,Adenocarcinoma,1,0.504307,Male,1,Current,0
997,Sandy2_AMC-032,adenocarcinoma,1,-1.302457,Female,0,Nonsmoker,1
998,CT_0822,Squamous cell carcinoma,0,1.090594,Male,1,Former,0


In [ ]:
print('##### TRAINING ######')
print("'''''''''''''''''''''")
print('Squamous    : ',df_train[df_train['Histology012']==0].shape[0])
print('Adeno : ',df_train[df_train['Histology012']==1].shape[0])
print('other    : ',df_train[df_train['Histology012']==2].shape[0])
print()
print()
print('##### Validation ######')
print("'''''''''''''''''''''")
print('Squamous    : ',df_val[df_val['Histology012']==0].shape[0])
print('Adeno : ',df_val[df_val['Histology012']==1].shape[0])
print('other    : ',df_val[df_val['Histology012']==2].shape[0])
print()
print()
print('##### Test ######')
print("'''''''''''''''''''''")
print('Squamous    : ',df_test[df_test['Histology012']==0].shape[0])
print('Adeno : ',df_test[df_test['Histology012']==1].shape[0])
print('other    : ',df_test[df_test['Histology012']==2].shape[0])

In [ ]:
print('##### TRAINING ######')
print("'''''''''''''''''''''")
print('Female    : ',df_train[df_train['gender']==0].shape[0])
print('Male : ',df_train[df_train['gender']==1].shape[0])
print('other    : ',df_train[df_train['gender']==2].shape[0])
print()
print()
print('##### Validation ######')
print("'''''''''''''''''''''")
print('Female    : ',df_val[df_val['gender']==0].shape[0])
print('Male : ',df_val[df_val['gender']==1].shape[0])
print('other    : ',df_val[df_val['gender']==2].shape[0])
print()
print()
print('##### Test ######')
print("'''''''''''''''''''''")
print('Female    : ',df_test[df_test['gender']==0].shape[0])
print('Male : ',df_test[df_test['gender']==1].shape[0])
print('other    : ',df_test[df_test['gender']==2].shape[0])

In [ ]:
print('##### TRAINING ######')
print("'''''''''''''''''''''")
print('Current,Former   : ',df_train[df_train['smoking']==0].shape[0])
print('Nonsmoker : ',df_train[df_train['smoking']==1].shape[0])
print('other    : ',df_train[df_train['smoking']==2].shape[0])
print()
print()
print('##### Validation ######')
print("'''''''''''''''''''''")
print('Current,Former   : ',df_val[df_val['smoking']==0].shape[0])
print('Nonsmoker : ',df_val[df_val['smoking']==1].shape[0])
print('other    : ',df_val[df_val['smoking']==2].shape[0])
print()
print()
print('##### Test ######')
print("'''''''''''''''''''''")
print('Current,Former   : ',df_test[df_test['smoking']==0].shape[0])
print('Nonsmoker : ',df_test[df_test['smoking']==1].shape[0])
print('other    : ',df_test[df_test['smoking']==2].shape[0])

In [ ]:
os.path.join(train_folder, 'train.csv')

In [ ]:
plt.plot([1,2,3],[1,2,3],'*')
plt.grid()

In [ ]:
a = Noneu
if a:
    print('i')

In [ ]:
np.log10(1e5)

In [36]:
a = [1]
b= [2,3]
a.append(b)

In [37]:
a

[1, [2, 3]]

In [39]:
a = [1]
b= [2,3]
for i in b:
    a.append(i)

In [40]:
a

[1, 2, 3]

In [41]:
[i+3 for i in b if i==2 else i+4]

SyntaxError: invalid syntax (tmp/ipykernel_8872/2645578089.py, line 1)

In [42]:
path = 'U:/Mori/Task501_MultiOrgan/imagesTs'
os.listdir(path)

['0001_Input_3D_0000.nii.gz',
 '0001_Post_Input_3D_0000.nii.gz',
 'CT_0011_0000.nii.gz',
 'CT_224693_0000.nii',
 'Decathlon_lung_001_ct_0000.nii.gz',
 'Hugo_LUNG1-001_ct_0000.nii.gz',
 'NSCLC_Hugo_89_pt_1_ct_0000.nii.gz',
 'Sandy1_R01-001_ct_0000.nii.gz',
 'Sandy2_AMC-002_ct_0000.nii.gz',
 'TCGA_LUAD_TCGA-17-Z011_ct_0000.nii.gz']

In [74]:

from collections import OrderedDict
json_dict = OrderedDict()

In [80]:
json_dict['training'] = [{'image': "./imagesTr/%s.nii.gz" % i[:-12],
                              "label": "./labelsTr/%s.nii.gz" % i[:-12]}
                               for i in os.listdir(path) if i.endswith('.gz')]

In [81]:
json_dict_training_nii = [{'image': "./imagesTr/%s.nii" % i[:-9],
                            "label": "./labelsTr/%s.nii" % i[:-9]}
                            for i in os.listdir(path) if i.endswith('.nii')]

In [69]:
json_dict['training'][0]

{'image': './imagesTr/0001_Input_3D.nii.gz',
 'label': './labelsTr/0001_Input_3D.nii.gz'}

In [70]:
a

[{'image': './imagesTr/CT_224693.nii', 'label': './labelsTr/CT_224693.nii.gz'}]

In [57]:
a[0]

{'image': './imagesTr/CT_224693.nii.gz',
 'label': './labelsTr/CT_224693.nii.gz'}

In [58]:
json_dict['training'].append(a)

In [82]:
for i in json_dict_training_nii:
    json_dict['training'].append(i)

In [83]:
json_dict['training']

[{'image': './imagesTr/0001_Input_3D.nii.gz',
  'label': './labelsTr/0001_Input_3D.nii.gz'},
 {'image': './imagesTr/0001_Post_Input_3D.nii.gz',
  'label': './labelsTr/0001_Post_Input_3D.nii.gz'},
 {'image': './imagesTr/CT_0011.nii.gz', 'label': './labelsTr/CT_0011.nii.gz'},
 {'image': './imagesTr/Decathlon_lung_001_ct.nii.gz',
  'label': './labelsTr/Decathlon_lung_001_ct.nii.gz'},
 {'image': './imagesTr/Hugo_LUNG1-001_ct.nii.gz',
  'label': './labelsTr/Hugo_LUNG1-001_ct.nii.gz'},
 {'image': './imagesTr/NSCLC_Hugo_89_pt_1_ct.nii.gz',
  'label': './labelsTr/NSCLC_Hugo_89_pt_1_ct.nii.gz'},
 {'image': './imagesTr/Sandy1_R01-001_ct.nii.gz',
  'label': './labelsTr/Sandy1_R01-001_ct.nii.gz'},
 {'image': './imagesTr/Sandy2_AMC-002_ct.nii.gz',
  'label': './labelsTr/Sandy2_AMC-002_ct.nii.gz'},
 {'image': './imagesTr/TCGA_LUAD_TCGA-17-Z011_ct.nii.gz',
  'label': './labelsTr/TCGA_LUAD_TCGA-17-Z011_ct.nii.gz'},
 {'image': './imagesTr/CT_224693.nii', 'label': './labelsTr/CT_224693.nii'}]

In [73]:
i

{'image': './imagesTr/CT_224693.nii', 'label': './labelsTr/CT_224693.nii.gz'}

In [84]:
json_dict['test'] = ["./imagesTs/%s.nii.gz" % i[:-12] for i in os.listdir(path) if i.endswith('.gz')]
json_dict_test_nii = ["./imagesTs/%s.nii" % i[:-9] for i in os.listdir(path) if i.endswith('.nii')]
for i in json_dict_test_nii:
    json_dict['test'].append(i)

In [85]:
json_dict['test']

['./imagesTs/0001_Input_3D.nii.gz',
 './imagesTs/0001_Post_Input_3D.nii.gz',
 './imagesTs/CT_0011.nii.gz',
 './imagesTs/Decathlon_lung_001_ct.nii.gz',
 './imagesTs/Hugo_LUNG1-001_ct.nii.gz',
 './imagesTs/NSCLC_Hugo_89_pt_1_ct.nii.gz',
 './imagesTs/Sandy1_R01-001_ct.nii.gz',
 './imagesTs/Sandy2_AMC-002_ct.nii.gz',
 './imagesTs/TCGA_LUAD_TCGA-17-Z011_ct.nii.gz',
 './imagesTs/CT_224693.nii']

In [86]:
os.getcwd()

'E:\\data\\Histology\\Nature_Gemini_4d_Feb2022'

In [96]:
from shutil import copyfile, copy
import os
from joblib import Parallel, delayed
import multiprocessing


def TCIA(i):
    path = ct_list_path[i]
    folder_single = path.split('\\')[-2]
    print(path)
    folder = os.path.join(des,folder_single)
    os.makedirs(folder, exist_ok=True)

    CT_tumor_window_source = os.path.join(path_SUV_Sheeba, folder_single, 'Seg_tumor.nii' )
    copy(CT_tumor_window_source, folder)
    os.remove(     os.path.join(folder,'CT_tumor_window.nii')    )



des = 'C:/Users/MSalehjahromi/Desktop/CT_PET_Segs'
os.makedirs(des, exist_ok=True)

path_SUV_Sheeba = 'E:/data/TCIA/CT_PET_Segs'
folders = [j for j in os.listdir(path_SUV_Sheeba)]
ct_list_path = [os.path.join(path_SUV_Sheeba,folder,'CT.nii') for folder in folders]
for i in range(len(ct_list_path)):
    TCIA(i)

E:/data/TCIA/CT_PET_Segs\AMC-004-1919102521447413\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-006-1529945160097891\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-010-6343011139620289\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-011-2525335627255871\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-013-1004610344524232\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-014-2270990546541356\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-015-9557494951655727\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-017-2088960115508162\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-020-2770155866932989\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-021-3500166790930934\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-023-3079787936364114\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-025-2605058705129812\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-026-2427994104521680\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-033-2895338545517030\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-035-2986195202342460\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-036-3286842444049692\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-037-8615479119108327\CT.nii
E:/data/TCIA/CT_PET_Segs\AMC-04

In [95]:
path = ct_list_path[0]
path.split('\\')[-2]

'AMC-004-1919102521447413'